## Data Wrangling with Python: Intro to Pandas
Note: Notebook adapted from [here](https://github.com/EricElmoznino/lighthouse_pandas_tutorial/blob/master/pandas_tutorial.ipynb) & [here](https://github.com/sedv8808/LighthouseLabs/tree/main/W02D2) & from LHL's [21 Day Data Challenge](https://data-challenge.lighthouselabs.ca/start)
#### Instructor: Simon Dawkins
#### Date: May 10, 2022

**Agenda:**
 - Why Pandas?
 - Pandas Basics
     - Pandas Series vs. Pandas DataFrames
     - .loc() vs. iloc()
 - Pandas Advance
     - Filtering
     - Group-bys
 - Pandas Exercises
     - Challenge 1
     - Challenge 2

### Pandas: Why Pandas? What is it? 


To do data anlaysis with Python, Pandas is a great tool to for dealing with data in a tabular and time series formats. Designed by Wes McKinney as an attempt to port R's dataframes to python. 

- Python Package for working with **tables**
- Similar to SQL & Excel
    - Faster sometimes - more optimized for non-distributed platforms
    - More features to manipulate, transform, and aggregate data
- Easy to handle messy and missing data
- Great at working with large datasets
- When combing with other Python libraries, it's fairly easy to create beautiful and customazied visuals. Easy integration with Matplotlib, Seaborn, Plotly.
- Easy integration with machine learning plugins (sckit-learn, etc)
    
    
-----------
To read more about, Wes McKinney, the creator of Pandas, check out the article below.

1. https://qz.com/1126615/the-story-of-the-most-important-tool-in-data-science/

--------------


## Think of how we would try to represent a table in Python?


In [3]:
#A dicitonary of lists example
students = {
    'student_id': [1, 2, 3, 4,5,6],
    'name': ['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'],
    'course_mark': [82, 100, 12, 76, 46, 20],
    'species': ['cat', 'human', 'cat', 'human', 'human', 'human']
}

**What are some operations we might want to do on this data?**

- 1. Select a subset of columns
- 2. Filter out some rows based on an attribute
- 3. Group by some attribute
- 4. Compute some aggregate values within groups
- 5. Save to a file

How about we try out one of these to see how easy it is

### Return a table with the mean course mark per-species with vanilla Python

In [2]:
# Return a table with the mean course mark per species

species_sums = {} # Tables of Sums
species_counts = {} # Count per Species
for i in range(len(students['species'])):  # iterating over the rows
    species = students['species'][i] # every row number I get species 
    course_mark = students['course_mark'][i] #  and course mark
    if species not in species_sums: # Intializing Species if not in list
        species_sums[species] = 0
        species_counts[species] = 0
    species_sums[species] += course_mark # Add each course mark for each species
    species_counts[species] += 1 

species_means = {}
                                  
for species in species_sums: # for every unique species we found
    species_means[species] = species_sums[species] / species_counts[species] #sum/count

species_means

{'cat': 47.0, 'human': 60.5}

- Did you like looking at this? 
- Does this look fun to do? 
- what about with 10000 rows and 50 columns?
- Super Tiring.

## Pandas Version

In [4]:
import pandas as pd

# Can take in a dictionry of list to instantiate a DataFrame
students = pd.DataFrame(students)
students

,student_id,name,course_mark,species
0,1,Daenerys,82,cat
1,2,Jon,100,human
2,3,Arya,12,cat
3,4,Sansa,76,human
4,5,Eddard,46,human
5,6,Khal Drogo,20,human


In [5]:
species_means = students[['species', 'course_mark']].groupby('species').mean()
# species_means = students.groupby('species')['course_mark'].mean()
species_means

,course_mark
species,
cat,47.0
human,60.5


### Dissecting the above code:


In [7]:
#Step 1: Filter out the columns we want to keep
students_filtered = students[['species','course_mark']]
students_filtered

,species,course_mark
0,cat,82
1,human,100
2,cat,12
3,human,76
4,human,46
5,human,20


In [8]:
# Step 2: Group by species column
students_grouped_by_species = students_filtered.groupby('species') 
students_grouped_by_species

In [9]:
#Step 3: Specify how to aggregate the course-mark column
species_means = students_grouped_by_species.mean()

In [10]:
species_means

,course_mark
species,
cat,47.0
human,60.5


#### As shown, Pandas makes use of vectorized operations. 


- Rather than use for-loops, we specify the operation that will apply to the structure as a whole (i.e. all the rows)
- By vectorizing, **the code becomes more concise and more readable**
- Pandas is optimized for vectorized operations (parallel vs. serial computation), which makes them **much faster**
- It is almost always possible to vectorize operations on Pandas data types - series have a single data type


### Getting Started: Pandas Series & Pandas DataFrames

There are two Pandas data types of interest:

- Series (column)
    - A pandas series is similar to an array but it has an index. The index is constant, and doesnt change through the operations we apply to the series. 
- DataFrame (table)
    - A pandas dataframe is an object that is similar to a collection of pandas series.

* series can have an index - still one dimensional, there's still an implicit integer index

In [11]:
# One way to construct a Series
grades = pd.Series([82, 100, 12, 76, 46, 20]) 
grades

0     82
1    100
2     12
3     76
4     46
5     20
dtype: int64

In [12]:
#We can specify some index when building a series. 
grades = pd.Series([82, 100, 12, 76, 46, 20], 
                   index = ['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'] ) 

grades

Daenerys       82
Jon           100
Arya           12
Sansa          76
Eddard         46
Khal Drogo     20
dtype: int64

In [11]:
print("The values:", grades.values)
print("The indexes:", grades.index)

The values: [ 82 100  12  76  46  20]
The indexes: Index(['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'], dtype='object')


**Note:** The underlying index is still 0, 1, 2, 3.... and we can still index on that:

In [12]:
grades[2]

12

### Pandas DataFrames

* data frame has bunch of series/columns, index is associated with all the columns

In [13]:
# One way to construct a DataFrame
df = pd.DataFrame({
    'name': ['Daenerys', 'Jon', 'Arya', 'Sansa'],
    'course_mark': [82, 100, 12, 76],
    'species': ['human', 'human', 'cat', 'human']},
    index=[1412, 94, 9351, 14])
df

,name,course_mark,species
1412,Daenerys,82,human
94,Jon,100,human
9351,Arya,12,cat
14,Sansa,76,human


#### Reading a CSV file

We'll use the function `read_csv()` to load the data into our notebook

- The `read_csv()` function can read data from a locally saved file or from a URL
- We'll store the data as a variable `df_pokemon`

In [13]:
pokemon = pd.read_csv('pokemon.csv')

In [14]:
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


**What do we see here?**
- Each row of the table is an observation, containing data of a single pokemon

In [17]:
# shape
# useful for certain algorithms that need specific inputs
pokemon.shape

(800, 13)

For large DataFrames, it's often useful to display just the first few or last few rows:

In [18]:
# head
pokemon.head(5)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [19]:
# tail
pokemon.tail(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
790,714,Noibat,Flying,Dragon,245,40,30,35,45,40,55,6,False
791,715,Noivern,Flying,Dragon,535,85,70,80,97,80,123,6,False
792,716,Xerneas,Fairy,NaN,680,126,131,95,131,98,99,6,True
793,717,Yveltal,Dark,Flying,680,126,131,95,131,98,99,6,True
794,718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


## Data at a Glance

`pandas` provides many ways to quickly and easily summarize your data:
- How many rows and columns are there?
- What are all the column names and what type of data is in each column?
- How many values are missing in each column or row?
- Numerical data: What is the average and range of the values?
- Text data: What are the unique values and how often does each occur?

### Peeking into the pokemon dataset

- Similar with getting familar with SQL tables, it is often a great idea to look at the pandas dataframes we are working with. Below are some of the basic methods to glance at a dataset. 

In [20]:
# Getting the Columns
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [21]:
# Getting Summary Statistics
pokemon.describe()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,362.813750,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,208.343798,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.000000,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,184.750000,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,364.500000,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,539.250000,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,721.000000,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


In [24]:
pokemon[['Total','HP']]
# allows for multiple columns

,Total,HP
0,318,45
1,405,60
2,525,80
3,625,80
4,309,39
...,...,...
795,600,50
796,700,50
797,600,80
798,680,80


In [23]:
# accessing columns with dot, but only one column and not defined column names
pokemon.Total

0      318
1      405
2      525
3      625
4      309
      ... 
795    600
796    700
797    600
798    680
799    600
Name: Total, Length: 800, dtype: int64

In [31]:
# Checking for Missing Data
pokemon.isnull().sum()

# as percentage 
# pokemon.isnull().sum() / len(pokemon)

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [32]:
# getting info
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


## The .loc() vs .iloc() method


To select rows and columns at the same time, we use the syntax `.loc[ROWS SLICE, COLUMNS SLICE]`:

In [40]:
# simple slice
# Notice the square brackets on loc, .loc is a method but uses [] instead of ()
# inclusive on the end point *** different from normal python slicing
# pokemon['Name'].loc[20:30]

# Other way
pokemon.loc[20:30, ['Attack']]

,Attack
20,45
21,60
22,80
23,80
24,56
25,81
26,60
27,90
28,60
29,85


In [39]:
# Getting more than one column
pokemon.loc[20:30, ['Attack','Legendary']]

,Attack,Legendary
20,45,False
21,60,False
22,80,False
23,80,False
24,56,False
25,81,False
26,60,False
27,90,False
28,60,False
29,85,False


In [27]:
# pass a list for the rows
pokemon.loc[[1,5,4,20,30], ['Attack','Defense']]

,Name,Legendary
1,Ivysaur,False
8,CharizardMega Charizard Y,False
21,Pidgeotto,False
7,CharizardMega Charizard X,False
432,Turtwig,False


In [41]:
# pass a range range of column values
# WHY it indexes inclusive with end point to include column names. 
# using columns at the end of row is more readable, and allows for complex slicing, can do steps, and range
pokemon.loc[1:7, 'Name':'HP']

,Name,Type 1,Type 2,Total,HP
1,Ivysaur,Grass,Poison,405,60
2,Venusaur,Grass,Poison,525,80
3,VenusaurMega Venusaur,Grass,Poison,625,80
4,Charmander,Fire,NaN,309,39
5,Charmeleon,Fire,NaN,405,58
6,Charizard,Fire,Flying,534,78
7,CharizardMega Charizard X,Fire,Dragon,634,78


In [65]:
#iloc is used for integer based indexing

pokemon.iloc[0:7, 0:3]

,#,Name,Type 1
0,1,Bulbasaur,Grass
1,2,Ivysaur,Grass
2,3,Venusaur,Grass
3,3,VenusaurMega Venusaur,Grass
4,4,Charmander,Fire
5,5,Charmeleon,Fire
6,6,Charizard,Fire


### Modifying a Column or Creating a new column

In [30]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [51]:
# Make sure make a copy of your dataframe before doing any modification
# This is generaly best practice, and it will be easier for you to keep track of the various
# changes and states of your dataframe. 

# make a soft copy, but independent copy in your memory - can be problematic with large data - 2 GB becomes 4 GB 
pokemon2 = pokemon.copy()

In [53]:
# create a new column
# Combine Attack + Special Attack
pokemon2['Total Attack'] = pokemon2['Attack'] + pokemon2['Sp. Atk']
pokemon2.head(2)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total Attack
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,114
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,142


In [54]:
# modify existing column
pokemon2['Total'] = pokemon2['Total'] * 2
pokemon2.head(2)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total Attack
0,1,Bulbasaur,Grass,Poison,636,45,49,49,65,65,45,1,False,114
1,2,Ivysaur,Grass,Poison,810,60,62,63,80,80,60,1,False,142


In [55]:
# Modify Data Frame with .loc() method
pokemon2.loc[1:5, ['Name']] = 'Changed'
pokemon2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total Attack
0,1,Bulbasaur,Grass,Poison,636,45,49,49,65,65,45,1,False,114
1,2,Changed,Grass,Poison,810,60,62,63,80,80,60,1,False,142
2,3,Changed,Grass,Poison,1050,80,82,83,100,100,80,1,False,182
3,3,Changed,Grass,Poison,1250,80,100,123,122,120,80,1,False,222
4,4,Changed,Fire,NaN,618,39,52,43,60,50,65,1,False,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,1200,50,100,150,100,150,50,6,True,200
796,719,DiancieMega Diancie,Rock,Fairy,1400,50,160,110,160,110,110,6,True,320
797,720,HoopaHoopa Confined,Psychic,Ghost,1200,80,110,60,150,130,70,6,True,260
798,720,HoopaHoopa Unbound,Psychic,Dark,1360,80,160,60,170,130,80,6,True,330


### Sort_values() & value_counts()

1. ***df.sort_values()***
2. ***df.value_counts()***


The ***pandas.sort_values()*** allows us to reorder our dataframe in an ascending or descending order given a column for pandas to work from. This is similar to the excel sort function.

```python
import pandas as pd
df = pd.read_csv('random.csv')
df


df.sort_values(by=['some_column'], ascending = True)
```
In the above code snippet, we are sorting our *random.csv* pandas data frame by the column *some_column* in ascending order. To read more on the ***df.sort_values()*** function, read this [article](https://datatofish.com/sort-pandas-dataframe/).

The second function is ***df.value_counts()***, it allows us to count how many times a specific value/item occurred in the dataframe. This function is best used on a specific column on a data frame, ideally on a column representing categorical data. Categorical data refers to a statistical data type consisting of categorical variables. 

```python
df['column'].value_counts()
```

To read more on some of the advanced functionalities of ***df.value_counts()***, please refer to the pandas documentation or this [article](https://towardsdatascience.com/getting-more-value-from-the-pandas-value-counts-aa17230907a6).

In [66]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [68]:
# Sort values 
# ascending = True default 
pokemon.sort_values(by="Generation", ascending=False)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True
738,670,Floette,Fairy,NaN,371,54,45,47,75,98,52,6,False
740,672,Skiddo,Grass,NaN,350,66,65,48,62,57,52,6,False
741,673,Gogoat,Grass,NaN,531,123,100,62,97,81,68,6,False
742,674,Pancham,Fighting,NaN,348,67,82,62,46,48,43,6,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,102,Exeggcute,Grass,Psychic,325,60,40,80,60,45,40,1,False
109,101,Electrode,Electric,NaN,480,60,50,70,80,80,140,1,False
108,100,Voltorb,Electric,NaN,330,40,30,50,55,55,100,1,False
107,99,Kingler,Water,NaN,475,55,130,115,50,50,75,1,False


In [74]:
# the first index take precedence 
pokemon.sort_values(by=['HP','Attack'], ascending=[False, True])

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,2,False
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,1,False
217,202,Wobbuffet,Psychic,NaN,405,190,33,58,33,58,33,2,False
351,321,Wailord,Water,NaN,500,170,90,45,90,45,60,3,False
655,594,Alomomola,Water,NaN,470,165,75,80,40,45,65,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,439,Mime Jr.,Psychic,Fairy,310,20,25,45,70,90,60,4,False
186,172,Pichu,Electric,NaN,205,20,40,15,35,35,60,2,False
388,355,Duskull,Ghost,NaN,295,20,40,90,30,90,25,3,False
55,50,Diglett,Ground,NaN,265,10,55,25,35,45,95,1,False


In [78]:
# .value_counts() for specific values
pokemon['Generation'].value_counts()

1    166
5    165
3    160
4    121
2    106
6     82
Name: Generation, dtype: int64

In [81]:
# can do value counts for lists of columns also 
pokemon['Type 1'].value_counts().sort_index()

Bug          69
Dark         31
Dragon       32
Electric     44
Fairy        17
Fighting     27
Fire         52
Flying        4
Ghost        32
Grass        70
Ground       32
Ice          24
Normal       98
Poison       28
Psychic      57
Rock         44
Steel        27
Water       112
Name: Type 1, dtype: int64

In [84]:
# list Unique Values
pokemon['Type 1'].unique()

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

In [85]:
# How many unique Values
pokemon['Type 1'].nunique()

18

### How to Query or Filter Data with Conditions?

- We can extract specific data from our dataframe based on a specific condition. We will be using the syntax below. Pandas will return a subset of the dataframe based on the given condition. 

```python
df[<insert_condition>]
```

Conditions follow the generic boolean logic in Python. Below is a cheat sheet python boolean logic.

**Conditional Logic:** 

Conditional logic refers to the execution of different actions based on whether a certain condition is met. In programming, these conditions are expressed by a set of symbols called **Boolean Operators**. 

| Boolean Comparator | Example | Meaning                         |
|--------------------|---------|---------------------------------|
| >                  | x > y   | x is greater than y             |
| >=                 | x >= y  | x is greater than or equal to y |
| <                  | x < y   | x is less than y                |
| <=                 | x <= y  | x is less than or equal to y    |
| !=                 | x != y  | x is not equal to y             |
| ==                 | x == y  | x is equal to y                 |




In [126]:
# Step 1: Create a filter
# make sure to add parentheses otherwise will evaluate 
myFilter = ((pokemon['Total'] >= 500) & (pokemon['Generation'] == 1))

In [128]:
# Step 2: Apply Filter
pokemon[myFilter].head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False


In [99]:
# in one line, usually how you want to do it, can be done with multiple 
pokemon[(pokemon['Total'] >= 500) & (pokemon['Legendary'] == True)]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
156,144,Articuno,Ice,Flying,580,90,85,100,95,125,85,1,True
157,145,Zapdos,Electric,Flying,580,90,90,85,125,90,100,1,True
158,146,Moltres,Fire,Flying,580,90,100,90,125,85,90,1,True
162,150,Mewtwo,Psychic,NaN,680,106,110,90,154,90,130,1,True
163,150,MewtwoMega Mewtwo X,Psychic,Fighting,780,106,190,100,154,100,130,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


Alternativly we can use the pandas **.where()** function, which has the following syntax.


```python 
df.where(<condition>, <What to fill inplace where the condition is not True>) # default is Nan
```

In [100]:
pokemon.where(myFilter, "Weak Pokemon")
# usually combined with a slice 

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon
1,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon,Weak Pokemon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


### Grouping and Aggregation 

Grouping and aggregation can be used to calculate statistics on groups in the data.

**Common Aggregation Functions**
- mean()
- median()
- sum()
- count()


In [101]:
# simple groupby
pokemon.groupby('Type 1').mean()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
Type 1,,,,,,,,,,
Bug,334.492754,378.927536,56.884058,70.971014,70.724638,53.869565,64.797101,61.681159,3.217391,0.000000
Dark,461.354839,445.741935,66.806452,88.387097,70.225806,74.645161,69.516129,76.161290,4.032258,0.064516
Dragon,474.375000,550.531250,83.312500,112.125000,86.375000,96.843750,88.843750,83.031250,3.875000,0.375000
Electric,363.500000,443.409091,59.795455,69.090909,66.295455,90.022727,73.704545,84.500000,3.272727,0.090909
Fairy,449.529412,413.176471,74.117647,61.529412,65.705882,78.529412,84.705882,48.588235,4.117647,0.058824
Fighting,363.851852,416.444444,69.851852,96.777778,65.925926,53.111111,64.703704,66.074074,3.370370,0.000000
Fire,327.403846,458.076923,69.903846,84.769231,67.769231,88.980769,72.211538,74.442308,3.211538,0.096154
Flying,677.750000,485.000000,70.750000,78.750000,66.250000,94.250000,72.500000,102.500000,5.500000,0.500000
Ghost,486.500000,439.562500,64.437500,73.781250,81.187500,79.343750,76.468750,64.343750,4.187500,0.062500


- By default, `groupby()` assigns the variable that we're grouping on (in this case `Type 1`) to the index of the output data
- If we use the keyword argument `as_index=False`, the grouping variable is instead assigned to a regular column
  - This can be useful in some situations, such as data visualization functions which expect the relevant variables to be in columns rather than the index

In [104]:
# grouby with as_index set to False, better for graphing - can easier grab
pokemon.groupby('Type 1', as_index = False).mean()

,Type 1,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bug,334.492754,378.927536,56.884058,70.971014,70.724638,53.869565,64.797101,61.681159,3.217391,0.000000
1,Dark,461.354839,445.741935,66.806452,88.387097,70.225806,74.645161,69.516129,76.161290,4.032258,0.064516
2,Dragon,474.375000,550.531250,83.312500,112.125000,86.375000,96.843750,88.843750,83.031250,3.875000,0.375000
3,Electric,363.500000,443.409091,59.795455,69.090909,66.295455,90.022727,73.704545,84.500000,3.272727,0.090909
4,Fairy,449.529412,413.176471,74.117647,61.529412,65.705882,78.529412,84.705882,48.588235,4.117647,0.058824
5,Fighting,363.851852,416.444444,69.851852,96.777778,65.925926,53.111111,64.703704,66.074074,3.370370,0.000000
6,Fire,327.403846,458.076923,69.903846,84.769231,67.769231,88.980769,72.211538,74.442308,3.211538,0.096154
7,Flying,677.750000,485.000000,70.750000,78.750000,66.250000,94.250000,72.500000,102.500000,5.500000,0.500000
8,Ghost,486.500000,439.562500,64.437500,73.781250,81.187500,79.343750,76.468750,64.343750,4.187500,0.062500
9,Grass,344.871429,421.142857,67.271429,73.214286,70.800000,77.500000,70.428571,61.928571,3.357143,0.042857


We can use the `agg` method to compute multiple aggregate functions on our data, for example minimum, maximum and mean attack in Type 1

In [105]:
# combine multiple agg functions
# can specify the column 
# more readable than pokemon['Attack']
pokemon.groupby('Type 1', as_index = False)['Attack'].mean()

,Type 1,Attack
0,Bug,70.971014
1,Dark,88.387097
2,Dragon,112.125000
3,Electric,69.090909
4,Fairy,61.529412
5,Fighting,96.777778
6,Fire,84.769231
7,Flying,78.750000
8,Ghost,73.781250
9,Grass,73.214286


In [113]:
# multi index - hard to work with - usually only as final output 
pokemon.groupby(['Type 1', 'Legendary']).mean().head()

#       Total          HP      Attack     Defense  \
Type 1 Legendary                                                               
Bug    False      334.492754  378.927536   56.884058   70.971014   70.724638   
Dark   False      451.517241  432.344828   64.655172   86.862069   68.689655   
       True       604.000000  640.000000   98.000000  110.500000   92.500000   
Dragon False      447.350000  476.850000   72.650000  103.400000   78.150000   
       True       519.416667  673.333333  101.083333  126.666667  100.083333   

                     Sp. Atk     Sp. Def       Speed  Generation  
Type 1 Legendary                                                  
Bug    False       53.869565   64.797101   61.681159    3.217391  
Dark   False       70.620690   67.827586   73.689655    3.965517  
       True       133.000000   94.000000  112.000000    5.000000  
Dragon False       72.900000   77.400000   72.350000    3.750000  
       True       136.750000  107.916667  100.833333    4.083333

We can also use `agg` to compute different agg functions for different columns:

In [115]:
# to specify with multiple agg methods 
pokemon.groupby('Type 1', as_index=False).agg(['min','max'])

/var/folders/fm/m7sbfrtn37dcz2lvxhvj6q0m0000gn/T/ipykernel_12200/715708997.py:2: FutureWarning: ['Type 2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  pokemon.groupby('Type 1', as_index=False).agg(['min','max'])


#                       Name                        Total       \
          min  max                   min                    max   min  max   
Type 1                                                                       
Bug        10  666              Accelgor                Yanmega   194  600   
Dark      197  717                 Absol               Zweilous   220  680   
Dragon    147  718               Altaria       Zygarde50% Forme   300  780   
Electric   25  702              Ampharos              Zebstrika   205  610   
Fairy      35  716            Aromatisse                Xerneas   218  680   
Fighting   56  701            Conkeldurr                Tyrogue   210  625   
Fire        4  721              Arcanine                 Vulpix   250  680   
Flying    641  715                Noibat  TornadusTherian Forme   245  580   
Ghost      92  711               Banette                 Yamask   275  680   
Grass       1  673             Abomasnow             Whimsicott   180  630   
Ground     27  645                Baltoy                Vibrava   265  770   
Ice       124  713              Articuno                Walrein   250  580   
Normal     16  676                 Aipom              Zigzagoon   190  720   
Poison     23  691                 Arbok                  Zubat   245  535   
Psychic    63  720                  Abra                   Xatu   198  780   
Rock       74  719            Aerodactyl                 Tyrunt   280  700   
Steel     208  707  AegislashBlade Forme    SteelixMega Steelix   300  700   
Water       7  693             Alomomola                 Wooper   200  770   

          HP      Attack       ... Sp. Atk      Sp. Def      Speed       \
         min  max    min  max  ...     min  max     min  max   min  max   
Type 1                         ...                                        
Bug        1   86     10  185  ...      10  135      20  230     5  160   
Dark      35  126     50  150  ...      30  140      30  130    20  125   
Dragon    41  125     50  180  ...      30  180      30  150    40  120   
Electric  20   90     30  123  ...      35  165      32  110    35  140   
Fairy     35  126     20  131  ...      40  131      40  154    15   99   
Fighting  30  144     35  145  ...      20  140      30  110    25  118   
Fire      38  115     30  160  ...      15  159      40  154    20  126   
Flying    40   85     30  115  ...      45  125      40   90    55  123   
Ghost     20  150     30  165  ...      30  170      33  135    20  130   
Grass     30  123     27  132  ...      24  145      30  129    10  145   
Ground    10  115     40  180  ...      20  150      30  120    10  120   
Ice       36  110     30  130  ...      30  130      30  200    25  110   
Normal    30  255      5  160  ...      15  135      20  135     5  135   
Poison    35  105     43  106  ...      30  100      40  123    25  130   
Psychic   20  190     20  190  ...      23  194      20  160    20  180   
Rock      30  123     40  165  ...      10  160      25  150    10  150   
Steel     40  100     24  150  ...      24  150      37  150    23  110   
Water     20  170     10  155  ...      10  180      20  160    15  122   

         Generation     Legendary         
                min max       min    max  
Type 1                                    
Bug               1   6     False  False  
Dark              2   6     False   True  
Dragon            1   6     False   True  
Electric          1   6     False   True  
Fairy             1   6     False   True  
Fighting          1   6     False  False  
Fire              1   6     False   True  
Flying            5   6     False   True  
Ghost             1   6     False   True  
Grass             1   6     False   True  
Ground            1   5     False   True  
Ice               1   6     False   True  
Normal            1   6     False   True  
Poison            1   6     False  False  
Psychic           1   6     False   True  
Rock              1   6

In [110]:
# to get specific agg functions on certain ones 
agg_dict = {
    'Attack' : 'mean',
    'Defense': ['min', 'max']
}

In [116]:
pokemon.groupby(['Type 1', 'Legendary'], as_index = False).agg(agg_dict).head()

Type 1 Legendary      Attack Defense     
                           mean     min  max
0     Bug     False   70.971014      30  230
1    Dark     False   86.862069      30  125
2    Dark      True  110.500000      90   95
3  Dragon     False  103.400000      35  130
4  Dragon      True  126.666667      80  121

### Challenge 1 (20 minutes)

Let's play around with Pandas on a more intricate dataset: a dataset on wines!

**Challenge 14 from the 21 Day Data Challenge** 

Dot's neighbour said that he only likes wine from Stellenbosch, Bordeaux, and the Okanagan Valley, and that the sulfates can't be that high. The problem is, Dot can't really afford to spend tons of money on the wine. Dot's conditions for searching for wine are: 
1. Sulfates cannot be higher than 0.6. 
2. The price has to be less than  $20. 

Use the above conditions to filter the data for questions **2 and 3** below. 

**Questions:**
1. Where is Stellenbosch, anyway? How many wines from Stellenbosch are there in the *entire dataset*? 
2. *After filtering with the 2 conditions*, what is the average price of wine from the Bordeaux region? 
3. *After filtering with the 2 conditions*, what is the least expensive wine that's of the highest quality from the Okanagan Valley?



**Stretch Question:**
1. What is the average price of wine from Stellenbosch, according to the entire unfiltered dataset? 


**Note: Check the dataset to see if there are missing values; if there are, fill in missing values with the mean.**


In [121]:
# code here
wineQual = pd.read_csv('winequality-red_2.csv')
wineQual = wineQual.drop(columns = ['Unnamed: 0'])
wineQual.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,region,price
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Colchagua Valley,64
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,Bordeaux,89
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,La Rjoja,25
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,Willamette,27
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Marlborough,9


In [131]:
wineQual['region'].value_counts()

La Rjoja            341
Bordeaux            264
Colchagua Valley    260
Okanagan Valley     256
Willamette          233
Marlborough         210
Stellenbosch         35
Name: region, dtype: int64

In [138]:
wineQualFilt = wineQual[(wineQual['sulphates'] <= 0.6) & (wineQual['price'] < 20)]
wineQualFilt.groupby('region', as_index = False)['price'].mean()

,region,price
0,Bordeaux,11.300000
1,Colchagua Valley,13.200000
2,La Rjoja,12.241379
3,Marlborough,9.333333
4,Okanagan Valley,10.400000
5,Stellenbosch,17.333333
6,Willamette,11.647059


In [145]:
wineQualFiltOka = wineQualFilt[wineQualFilt['region']=='Okanagan Valley']
wineQualFiltOka.groupby('price').min().sort_values('quality', ascending = False)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,region
price,,,,,,,,,,,,,
9,8.8,0.61,0.19,4.00,0.094,30.0,69.0,0.99787,3.22,0.50,10.0,6,Okanagan Valley
13,10.0,0.56,0.24,2.20,0.079,19.0,58.0,0.99910,3.18,0.56,10.1,6,Okanagan Valley
4,7.9,0.49,0.00,1.90,0.082,17.0,43.0,0.99680,3.20,0.55,9.5,5,Okanagan Valley
5,6.3,0.18,0.00,0.90,0.039,5.0,13.0,0.99007,2.89,0.44,10.4,5,Okanagan Valley
10,7.4,0.66,0.00,1.80,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5,Okanagan Valley
11,5.9,0.46,0.00,1.90,0.077,25.0,44.0,0.99385,3.50,0.53,11.2,5,Okanagan Valley
15,11.1,0.35,0.48,3.10,0.090,5.0,21.0,0.99860,3.17,0.53,10.5,5,Okanagan Valley
17,6.1,0.32,0.06,1.80,0.086,5.0,32.0,0.99464,3.36,0.44,10.1,5,Okanagan Valley
18,6.4,0.69,0.00,1.65,0.055,7.0,12.0,0.99162,3.41,0.53,9.4,5,Okanagan Valley


### Challenge 2 (25 minutes)

**Challenge 21 from the 21DDC (Adapted)**

Dot wants to play retro video games with all their new friends! Help them figure out which games would be best.

Questions: 
    
1. What is the top 5 best selling games released before the year 2000.

     -  **Note**: Use Global_Sales
    
    
2. Create a new column called Aggregate_Score, which returns the proportional average between Critic Score and User_Score based on Critic_Count and User_Count. Plot a horizontal bar chart of the top 5 highest rated games by Aggregate_Score, not published by Nintendo before the year 2000. From this bar chart, what is the highest rated game by Aggregate_Score?

    -  **Note**: Critic_Count should be filled with the mean. User_Count should be filled with the median.
    
    
#### In the exercise above, there is some missing values in the dataset. Look up the pandas documentation to figure out how to fill missing values in a column. You will be using the **fillna()** function.   

In [76]:
# code here

# HINT

**How to create the Aggregate Score Column?**

\begin{equation*}
AggregateScore = \frac{(CriticCount * CriticScore)+(UserCount * UserScore)}{UserCount + CriticCount}
\end{equation*}

**Check Your Column Values**

The Critic_Score column is scored out of 100. The User_Score column is scored out of 10. You will need to modify one of the columns to match the other.

## Documentation

In the meantime, check out pandas the user guide in the [pandas documentation](https://pandas.pydata.org/docs/user_guide/index.html#user-guide).

-------
**Why should I use the documentation?**

On the job as a data scientist or data analyst, more often than not, you may find yourself looking up the documentation of a particular function or plugin you use. Don't worry if there are a few functions you don't know by heart. However, there are just too many to know! An essential skill is to learn how to navigate documentation and understand how to apply the examples to your work. 

--------

Additional resources:

- To learn more about these topics, as well as other topics not covered here (e.g. reshaping, merging, additional subsetting methods, working with text data, etc.) check out [these introductory tutorials](https://pandas.pydata.org/docs/getting_started/index.html#getting-started) from the `pandas` documentation
- To learn more about subsetting your data, check out [this tutorial](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#min-tut-03-subset)
- This [pandas cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) may also be helpful as a reference.